# BioPython
Biopython is a large python module for performing common procedures in bioinformatics, including e.g. analysis of sequences.

http://biopython.org/

# Retrieve a sequence from UniProt

In [ ]:
accession = 'Q9BV79'

## Annotations

In [ ]:
from Bio import ExPASy
from Bio import SwissProt

handle = ExPASy.get_sprot_raw(accession) # download data from UniProt
record = SwissProt.read(handle) # parse (interpret) the data

In [ ]:
print(record.description)

In [ ]:
print(help(record))

## Structures available?

In [ ]:
for cross_ref in record.cross_references:
    if cross_ref[0] == 'PDB':
        print(cross_ref)

In [ ]:
pdb_codes = []
for cross_ref in record.cross_references:
    if cross_ref[0] == 'PDB':
        pdb_codes.append(cross_ref[1])

print(pdb_codes)

## Sequence

In [ ]:
print(record.sequence)

In [ ]:
sequence = record.sequence
print('>', record.entry_name, record.accessions[0])
print(record.sequence)


## Multiple proteins

In [ ]:
accessions = ['P97584', 'P00328', 'P19096', 'S0DRI1', 'A2R6H1', 'Q29073', 'P34055', 'Q24K16', 'W7LKX1', 
              'P38230', 'Q4W4Z2', 'W7MT31', 'Q64413', 'Q9Z2M2', 'P00327', 'P49327', 'P12785', 'Q9SLN8',
              'A0A0D2YG10', 'P79896', 'P26646', 'P0DN30', 'F2Z678', 'Q9P6C8', 'O57380', 'P22797']

In [ ]:
for accession in accessions:
    handle = ExPASy.get_sprot_raw(accession) # download data from UniProt
    record = SwissProt.read(handle) # parse (interpret) the data
    pdb_codes = []
    for cross_ref in record.cross_references:
        if cross_ref[0] == 'PDB':
            pdb_codes.append(cross_ref[1])
    if len(pdb_codes) > 0: # if any structures were found
        print(accession)
        print(', '.join(pdb_codes))
        print()

# BLAST

In [ ]:
query_fasta = sequence
print(query_fasta)

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# Use BLAST at NCBI
result_handle = NCBIWWW.qblast("blastp", "swissprot", query_fasta)

# parse the BLAST result
blast_record = NCBIXML.read(result_handle)

In [ ]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
       print('****Alignment****')
       print('sequence:', alignment.title)
       print('length:', alignment.length)
       print('e value:', hsp.expect)

In [ ]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < 10**-20:
            print(alignment.title)

In [ ]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < 10**-20:
            print(alignment.title.split('|')[3])

In [ ]:
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < 10**-20:
            raw_accession = alignment.title.split('|')[3]
            print(raw_accession[:raw_accession.index('.')])

In [ ]:
blast_accessions = []

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < 10**-20:
            raw_accession = alignment.title.split('|')[3]
            blast_accessions.append(raw_accession[:raw_accession.index('.')])
    
    
print(blast_accessions)

In [ ]:
for accession in blast_accessions:
    handle = ExPASy.get_sprot_raw(accession) # download data from UniProt
    record = SwissProt.read(handle) # parse (interpret) the data
    pdb_codes = []
    for cross_ref in record.cross_references:
        if cross_ref[0] == 'PDB':
            pdb_codes.append(cross_ref[1])
    if len(pdb_codes) > 0: # if any structures were found
        print(accession)
        print(', '.join(pdb_codes))
        print()

# Multiple sequence alignment

## Get sequences

In [ ]:
fasta_sequences = []
for accession in blast_accessions:
    handle = ExPASy.get_sprot_raw(accession) # download data from UniProt
    record = SwissProt.read(handle)
    fasta_sequences.append('> ' + record.entry_name + ' ' + record.accessions[0] + '\n'
                            + record.sequence + '\n') # \n = newline
for sequence in fasta_sequences:
    print(sequence)

In [ ]:
outfile = open('blast_hits.fasta', 'w')
for sequence in fasta_sequences:
        outfile.write(sequence)

### Make multiple sequence alignment in Clustal Omega

In [ ]:
# using clustal omega externally
!clustalo -i blast_hits.fasta -o blast_hits.cali --force

### Continue with analysis

In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("blast_hits.cali", "fasta")

In [ ]:
for seq in alignment:
    print(seq.seq[:100])

In [ ]:
from Bio.Align import AlignInfo

summary_align = AlignInfo.SummaryInfo(alignment)

## Consensus sequence

In [ ]:
summary_align.dumb_consensus()

## PSSM

In [ ]:
pssm = summary_align.pos_specific_score_matrix()

print(pssm)

In [ ]:
for position in pssm:
    print(max(position.values())/len(alignment))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


values = []
for position in pssm:
    values.append(max(position.values())/len(alignment))
    
fig = plt.figure(figsize = (20, 10))
plt.plot(values)

# Assignment 1

## Handle sequences and perform alignment

In [ ]:
handle = ExPASy.get_sprot_raw('P08686')
record = SwissProt.read(handle)

cyp21_seq = ('> ' + record.entry_name + ' ' + record.accessions[0] + '\n'
                + record.sequence + '\n') # parantheses needed when making multiline string outside function

mut_seq = '''>cyp21a2_translated_mut
MLLLGLLLLLPLLAGARLLWNWWKLRSLHLPPLAPGFLHLLQPDLPIYLLGLTQKFGPIY
RLHLGLQDVVVLNSKRTIEEAMVKKWADFAGRPEPLTYRLVSKNYPDLSLGDYSLLWKAH
KKLTRSALLLGIRDSMEPVVEQLTQEFCERMRAQPGTPVAIEEEFSLLTCSIICYLTFGD
KIKDDNLMPAYYKCIQEVLKTWSHWSIQIVDVIPFLRFFPNPGLRRLKQAIEKRDHIVEM
QLRQHKESLVAGQWRDMMDYMLQGVAQPSMEEGSGQLLEGHVHMAAVDLLISGTETTANT
LSWAVVFLLHHPEIQQRLQEELDHELGPGASSSRVPYKDRARLPLLNATIAEVLRLRPVV
PLALPHRTTRPSSISGYDIPEGTVIIPNLQGAHLDETVWERPHEFWPDRFLEPGKNSRAL
AFGCGARVRLGEPLARLELFVVLTRLLQAFTLLPSGDALPSLQPLPHCSVILKMQPFQVR
LQPRGMGAHSPGQNQ'''

with open('cyp21_comparison.fasta', 'w') as outfile:
    outfile.write(cyp21_seq)
    outfile.write(mut_seq)

In [ ]:
# kalign used externally
! kalign cyp21_comparison.fasta > cyp21_comparison.kali 2> /dev/null

## Find mutations

In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("cyp21_comparison.kali", "fasta")

print('Pos\tCYP21\tMut')
muts = []
prot_pos = 0
positions = []
for i in range(len(alignment[0])):
    if alignment[0][i] != alignment[1][i]:
        muts.append([str(prot_pos+1), alignment[0][i], alignment[1][i]])
        positions.append(prot_pos+1)
        print('\t'.join(muts[-1]))
    if alignment[0][i] != '-':
        prot_pos += 1


## Look for information in UniProt

In [ ]:
handle = ExPASy.get_sprot_raw('P08686')
record = SwissProt.read(handle)

In [ ]:
for feature in record.features:
    if feature[1] in positions:
        print(feature)

In [ ]:
for feature in record.features:
    if feature[1] in positions:
        print('\t'.join([str(feature[1]), feature[3][:100]]))